In [ ]:
pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.0 MB/s 
     |████████████████████████████████| 46 kB 4.4 MB/s 
     |████████████████████████████████| 198 kB 50.8 MB/s 
     |████████████████████████████████| 1.3 MB 38.9 MB/s 
     |████████████████████████████████| 3.7 MB 67.4 MB/s 
     |████████████████████████████████| 98 kB 7.2 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 130 kB 76.0 MB/s 
     |████████████████████████████████| 3.5 MB 76.5 MB/s 
     |████████████████████████████████| 99 kB 9.1 MB/s 
     |████████████████████████████████| 1.2 MB 36.0 MB/s 
     |████████████████████████████████| 128 kB 63.5 MB/s 
     |████████████████████████████████| 1.6 MB 52.3 MB/s 
     |████████████████████████████████| 79 kB 8.8 MB/s 
  Created wheel for ee-extra: filename=ee_extra-0.0.14-py3-none-any.whl size=209116 sha256=cfbc38f17

In [ ]:
import os
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=JW8nwkVXuVj3zijurwcGRGOhZNuvTr4sPQ9CxuHw8Ug&tc=CWkxW3CLo3S0dBb4_vhWCRksCEL6zgoRHyUSGcpsHGs&cc=EmuupfL5Q2BAtWQB0-ysbml17FpzpSKL5TvY9WLMNsw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdcq2p80Mu-YSL-cX6_7dsA-Q2QecB4FVmQXzFPsMofHC7SZe7IRbQ8

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
terai_shp = ee.FeatureCollection('projects/iron-byte-355521/assets/terai_shp')
Map.centerObject(terai_shp,8)
Map.addLayer(terai_shp)

In [ ]:
collection = (
    ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
    .filterBounds(terai_shp)
    .filterDate('2001-01-01', '2001-12-30')
    .filterMetadata('CLOUD_COVER', 'less_than',10)
    .median()
    .select('SR_B[1-7]')
)
clipped = collection.clip(terai_shp)
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

  return image.addBands(srcImg = opticalBands, overwrite = True)\

dataset = applyScaleFactors(clipped)
vis_params = {'min': 0, 'max': 0.3, 'bands': ['SR_B3', 'SR_B2', 'SR_B1']}

Map.centerObject(terai_shp, 8)
Map.addLayer(dataset, vis_params, "Landsat-7")

Map

Map(bottom=27843.0, center=[27.572102994869983, 84.03899668290383], controls=(WidgetControl(options=['position…

In [ ]:
forest_shp = ee.FeatureCollection('projects/iron-byte-355521/assets/forestpoints')

Map.centerObject(forest_shp,8)
Map.addLayer(forest_shp)


In [ ]:
urban_shp = ee.FeatureCollection('projects/iron-byte-355521/assets/city')
Map.centerObject(urban_shp,8)
Map.addLayer(urban_shp)

In [ ]:
agriculture_shp = ee.FeatureCollection('projects/iron-byte-355521/assets/croplandpoints')
Map.centerObject(agriculture_shp,8)
Map.addLayer(agriculture_shp)

In [ ]:
water_shp = ee.FeatureCollection('projects/iron-byte-355521/assets/waterpoints')
Map.centerObject(water_shp,8)
Map.addLayer(water_shp)

In [ ]:
river_bed = ee.FeatureCollection('projects/iron-byte-355521/assets/riverbedpoints')
Map.centerObject(river_bed,8)
Map.addLayer(river_bed)

In [ ]:
baresoil_shp = ee.FeatureCollection('projects/iron-byte-355521/assets/baresoil_points')
Map.centerObject(baresoil_shp,8)
Map.addLayer(baresoil_shp)

In [ ]:
grassland_shp = ee.FeatureCollection('projects/iron-byte-355521/assets/grasslandpoints')
Map.centerObject(grassland_shp,8)
Map.addLayer(grassland_shp)

In [ ]:
OWL_shp = ee.FeatureCollection('projects/iron-byte-355521/assets/owl_points')
Map.centerObject(OWL_shp,8)
Map.addLayer(OWL_shp)

.set gives the property for the shapefile. It did not not work in this code, I changed from arcgis pro. To include the feature class, we need to have property.


In [ ]:
forest_shp = forest_shp.set('landcover',1)
urban_shp = urban_shp.set('landcover',2)
agriculture_shp = agriculture_shp.set('landcover',3)
water_shp = water_shp.set('landcover',4)
river_bed = river_bed.set('landcover',5)
baresoil_shp = baresoil_shp.set('landcover',6)
grassland_shp = grassland_shp.set('landcover',7)
OWL_shp = OWL_shp.set('landcover',8)

In [ ]:
class_names_a = forest_shp.merge(urban_shp).merge(agriculture_shp).merge(water_shp).merge(river_bed).merge(baresoil_shp).merge(grassland_shp).merge(OWL_shp)


In [ ]:
# Use these bands for prediction.
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = clipped.select(bands).sampleRegions(
    **{'collection': class_names_a, 'properties': [label], 'scale':30}
)



In [ ]:
withRandom = training.randomColumn()

In [ ]:
split = 0.7

In [ ]:
training_points = withRandom.filter(ee.Filter.lt('random',split))
testing_points = withRandom.filter(ee.Filter.gte('random',split))

In [ ]:
training_points.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '1_1_1_1_1_1_1_00000000000000000000_1',
 'properties': {'SR_B1': 8845,
  'SR_B2': 9764,
  'SR_B3': 10075,
  'SR_B4': 18672.5,
  'SR_B5': 17045.5,
  'SR_B7': 12748.5,
  'landcover': 8,
  'random': 0.07528776384278035}}

In [ ]:
testing_points.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '1_1_1_1_1_1_1_00000000000000000000_0',
 'properties': {'SR_B1': 8787,
  'SR_B2': 9619.5,
  'SR_B3': 9842,
  'SR_B4': 16114,
  'SR_B5': 17095,
  'SR_B7': 13232.5,
  'landcover': 8,
  'random': 0.8766671777795254}}

In [ ]:
# Train a CART classifier with default parameters.
trained = ee.Classifier.smileGradientTreeBoost().train(training_points, label, bands)

None


In [ ]:
train_accuracy = trained.confusionMatrix()
train_accuracy.getInfo()


[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 211, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 3502, 0, 0, 0, 0, 0, 3],
 [0, 0, 3, 3542, 0, 0, 0, 2, 6],
 [0, 0, 1, 0, 3489, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 1406, 3, 0, 0],
 [0, 0, 1, 0, 0, 0, 3477, 5, 0],
 [0, 0, 0, 0, 1, 1, 5, 3477, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 3471]]

In [ ]:
train_accuracy.accuracy().getInfo()

0.9984961740899642

In [ ]:
train_accuracy.kappa().getInfo()

0.9982357336545411

In [ ]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.9553041018387554],
 [0.9343334276818568],
 [0.9600912200684151],
 [0.905848787446505],
 [0.912166003411029],
 [0.927461139896373],
 [0.934931506849315]]

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  1,
  0.9206652126499455,
  0.9285513361462728,
  0.9036758787228334,
  0.9577677224736049,
  0.943546015877683,
  0.9632286995515695,
  0.9490150637311703]]

In [ ]:
validated = testing_points.classify(trained)

In [ ]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [ ]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 94, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 586, 282, 211, 12, 57, 26, 321],
 [0, 0, 335, 403, 205, 29, 95, 63, 317],
 [0, 0, 133, 137, 884, 55, 173, 54, 73],
 [0, 0, 32, 51, 255, 99, 71, 57, 24],
 [0, 0, 101, 114, 393, 45, 485, 268, 111],
 [0, 0, 88, 91, 236, 25, 274, 722, 77],
 [0, 0, 313, 246, 98, 9, 37, 25, 800]]

In [ ]:
test_accuracy.accuracy().getInfo()

EEException: ignored

In [ ]:
test_accuracy.kappa().getInfo()

0.07080597865821904

In [ ]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.04922644163150492],
 [0.044504383007417395],
 [0.2991967871485944],
 [0.11428571428571428],
 [0.22490221642764016],
 [0.38817998656816655],
 [0.9660954425031021]]

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9772727272727273,
  0.21148036253776434,
  0.1813186813186813,
  0.3350824587706147,
  0.23129251700680273,
  0.3670212765957447,
  0.5494296577946768,
  0.8594528476386527]]

In [ ]:
result = clipped.select(bands).classify(trained)

var_parameters = {

    'min': 0,
    'max': 10,
    'palette': ['fffbd0','009966','febaad','df1b05','f1e132','a8a0ea','8a4482','abfbe8']
}
# # Display the clusters with random colors.
Map.addLayer(result, var_parameters,'classfied')
Map

Map(bottom=27781.0, center=[27.873072565422785, 83.29833984375001], controls=(WidgetControl(options=['position…

In [ ]:
remap = ee.Image(result).remap([0,1,2,3,4,5,6,7,], [1,2,3,4,5,6,7,8])
Map.addLayer(remap)

In [ ]:
geemap.ee_export_image_to_drive(result, description='landcover_terai_2001_try_1', folder='export', region = terai_shp.geometry(), scale=30, maxPixels = 1e13)